In [1]:
from evaluation import JSONParseEvaluator

import os, glob, json
import argparse
import numpy as np
from tqdm import tqdm
from os.path import join, basename, splitext

In [2]:
field_list = [ 'current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt', 'diagnosis', 'date_in', 'doctor_name', 'drug_name', 'drug_dose', 'drug_quantity']

def convert_dict_concern_fileds(raw_dict: dict, concern_fileds=None, is_label=True):
    res = {}
    if concern_fileds is None:
        concern_fileds = list(raw_dict.keys())

    for field,data in raw_dict.items():
        field = field.replace(' ', '_')
        if field in concern_fileds:
            res[field] = data['value'].lower() if is_label else data.lower()
    return res


In [3]:
def convert_output(output):
    list = ['patient', 'medical']

    list_child = [['name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt'], ['current_institute', 'diagnosis', 'date_in', 'doctor_name']]
    list_drug = ['drug_name', 'drug_dose', 'drug_quantity']
    
    result = {}
    
    for i, name in enumerate(list):
        if name in output.keys():
            for name_child in list_child[i]:
                try:
                    result[name_child] = output[name][name_child]
                except:
                    result[name_child] = ''
    if 'drugs' in output.keys():
        for field in list_drug:
            value = ""
            for drug in output['drugs']:
                for key, val in drug.items():
                    if key == field:
                        value += val
            result[field] = value
    return result

In [4]:

path_mistral_1 = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct/Mistral-7B'
path_mistral_json_1 = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct/Mistral-7B-json'

path_labels = '/home/vinbig/Documents/PA_Modeling/Prompt/prescription_label_text/KIEs'

In [5]:
def get_outputs_labels(path_output, path_labels):
    names = os.listdir(path_output)
    name_txts = []
    preds = []
    labels = []
    for name in names:
        # check file json
        if name.split('.')[1] == 'txt':
            name_txts.append(name)
            continue

        # correct -> read output and label
        pred_path = os.path.join(path_output, name)
        with open(pred_path, 'r') as f:
            pred = json.load(f)
        #convert
        pred = convert_output(pred)

        label_path = os.path.join(path_labels, name)
        with open(label_path, 'r') as f:
            label = json.load(f)
        
        label = convert_dict_concern_fileds(label, concern_fileds=field_list)
        pred  = convert_dict_concern_fileds(pred, concern_fileds=field_list, is_label=False)

        labels.append(label)
        preds.append(pred)
    return preds, labels, name_txts

In [6]:
def compute_acc(preds, labels):
    evaluator = JSONParseEvaluator()
    accs = []
    for i in range (len(labels)):
        acc = evaluator.cal_acc(pred=preds[i], answer=labels[i])
        accs.append(acc)
    return np.mean(accs)

In [7]:
def get_each_field(field, input):
    result = {}
    try:
        result[field] = input[field]
    except:
        result[field] = ""
    return result

In [9]:
def compute(preds, labels, name_txts, field_list):
    print("Số lượng file đúng format: ", len(preds), '/249')
    acc_mean = []
    # all field
    acc = compute_acc(preds, labels)
    acc_mean.append(acc)

    #each field
    for field in field_list:
        preds_field = [get_each_field(field=field, input=pred) for pred in preds]
        labels_field = [get_each_field(field=field, input=label) for label in labels]
        acc_mean.append(compute_acc(preds_field, labels_field))

    return acc_mean

In [10]:
print('Mistral')
preds, labels, name_txts = get_outputs_labels(path_labels=path_labels, path_output=path_mistral_1)
acc_mean = compute(preds, labels, name_txts, field_list)

Mistral
Số lượng file đúng format:  244 /249


/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: invalid value encountered in scalar divide
  zss.distance(
/home/vinbig/Documents/PA_Modeling/Prompt/evaluation.py:221: RuntimeWarning: divide by zero encountered in scalar divide
  zss.distance(


In [11]:
print('Mistral Json')
preds_json, labels_json, name_txts_json = get_outputs_labels(path_labels=path_labels, path_output=path_mistral_json_1)
acc_mean_json = compute(preds_json, labels_json, name_txts_json, field_list)

Mistral Json
Số lượng file đúng format:  243 /249


In [12]:
import pandas as pd
model_name = ["mistral", "mistral_json"]
column_name_all = ['all', 'current_institute', 'name', 'gender', 'birth', 'age', 'address', 'tel_customer', 'id_bhyt', 'diagnosis', 'date_in', 'doctor_name', 'drug_name', 'drug_dose', 'drug_quantity']
df = pd.DataFrame([acc_mean, acc_mean_json], columns=column_name_all, index=model_name)
df

,all,current_institute,name,gender,birth,age,address,tel_customer,id_bhyt,diagnosis,date_in,doctor_name,drug_name,drug_dose,drug_quantity
mistral,0.722094,0.475479,0.949712,0.795355,0.451600,0.200538,0.860924,0.387338,0.289811,0.852713,0.689540,0.614229,0.737924,0.658216,0.707648
mistral_json,0.719617,0.471873,0.951033,0.794513,0.453459,0.209647,0.858016,0.384817,0.291004,0.852583,0.691536,0.614254,0.735374,0.654950,0.708915


In [12]:
import json
path = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma_out_full_En_struct/Mistral-7B/Long_Chau_509.json'
with open (path, 'r') as f:
    out = json.load(f)
out

{'patient': {'name': 'TA QUANG MINH (MR.)',
  'gender': '',
  'birth': '17 Oct 1985',
  'age': '37Y',
  'address': 'HN: 218/3 An Dương Vương, phường 16, Quận 8, Thành phố Hồ Chim',
  'tel_customer': '',
  'id_bhyt': ''},
 'medical': {'current_institute': 'BỆNH VIỆN Guardian:',
  'diagnosis': '[K74] FIBROSIS AND CIRRHOSIS OF LIVER',
  'date_in': '12 Dec 2022',
  'doctor_name': 'Deputy Head of Gastro-nntrrollyy and Hepatology'},
 'drugs': [{'drug_name': 'Mirtazapine (REMERON) tab 30mg',
   'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày vào buổi sáng trong 30 ngày',
   'drug_quantity': '08 Viên'},
  {'drug_name': 'Propranolol (PROPRANOLOL TV PHARM) tab 40mg',
   'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày lúc đi ngủ trong 30 ngày',
   'drug_quantity': '08 Viên'},
  {'drug_name': 'Thiamine (Vit B1) (VITAMIN B1) tab 250mg',
   'drug_dose': 'Mỗi lần uống 1 viên hai lần một ngày sau khi ăn trong 30 ngày',
   'drug_quantity': '60 Viên'},
  {'drug_name': '(LIVACT) 4.15 g/sachet

In [13]:
convert_output(out)

{'name': 'TA QUANG MINH (MR.)',
 'gender': '',
 'birth': '17 Oct 1985',
 'age': '37Y',
 'address': 'HN: 218/3 An Dương Vương, phường 16, Quận 8, Thành phố Hồ Chim',
 'tel_customer': '',
 'id_bhyt': '',
 'current_institute': 'BỆNH VIỆN Guardian:',
 'diagnosis': '[K74] FIBROSIS AND CIRRHOSIS OF LIVER',
 'date_in': '12 Dec 2022',
 'doctor_name': 'Deputy Head of Gastro-nntrrollyy and Hepatology',
 'drug_name': 'Mirtazapine (REMERON) tab 30mgPropranolol (PROPRANOLOL TV PHARM) tab 40mgThiamine (Vit B1) (VITAMIN B1) tab 250mg(LIVACT) 4.15 g/sachet oral granulesUrsodeoxycholic acid (URSOBIL 300) tab 300mg',
 'drug_dose': 'Mỗi lần uống 0.25 viên một lần mỗi ngày vào buổi sáng trong 30 ngàyMỗi lần uống 0.25 viên một lần mỗi ngày lúc đi ngủ trong 30 ngàyMỗi lần uống 1 viên hai lần một ngày sau khi ăn trong 30 ngàyMỗi lần uống ba lần một ngày sau khi ăn trong 30 ngàyMỗi lần uống 1 viên ba lần một ngày sau khi ăn trong 30 ngày',
 'drug_quantity': '08 Viên08 Viên60 Viên90 Viên90 Viên'}